<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Sentiment Analysis of Movie Reviews with Spacy and VADER

---

### Learning Objectives
- Understand the goal of basic sentiment analysis.
- Calculate sentiment scores manually using a reviews dataset and scores tagged by word.
- Practice using the spacy parser to get out part of speech tags from text.
- Fit a model using sentiment and grammar features.
- Use the VADER sentiment analyzer to get out more accurate sentiment scores and compare the models.

### Lesson Guide
- [Introduction to sentiment analysis](#intro)
- [Load the word sentiment dataset](#load-sen)
    - [Engineer objectivity and positive difference scores](#adj-scores)
    - [Put scores in a part of speech dictionary](#pos-dict)
- [Load the rotten tomatoes review dataset](#rt-reviews)
    - [Restrict reviews to valid lengths and ratings](#subset)
- [Import spacy](#spacy)
    - [Parse all the quotes using spacy's multithreaded parser](#multi)
- [Part of speech features](#pos-features)
- [Assign sentiment scores](#assign)
- [Print out the most positive and most negative reviews](#print-most)
- [Print out the most objective and most subjective reviews](#print-most-obj)
- [Build a model to classify fresh vs. rotten with the sentiment and grammar features](#model)
- [User the VADER library to get better sentiment scores](#vader)
    - [Build a model using the VADER sentiment features](#vader-model)

<a id='intro'></a>

## Introduction to sentiment analysis
---

Sentiment analysis is one of the most popular topics in NLP. Most commonly it is the quantification of text into valence and subjectivity scores.

First we will load in a dataset of pre-coded sentiment scores for positivity and negativity on words. These words are also tagged with their part of speech in the sentence. We can use these valence scores to evaluate the sentiment of rottentomatoes movie reviews. Many packages such as TextBlob come pre-packaged with sentiment scores for words after parsing text, but doing the sentiment parsing manually will show you how it can be done without any "magic".

We will also explore a more advanced sentiment analysis library in python: [VADER](https://github.com/cjhutto/vaderSentiment). We can parse the sentiment of the movie reviews using this package and compare it to our more basic method.



<a id='load-sen'></a>

## Load the word sentiment dataset
---

Below we will load in some pre-tagged positive and negative valence scores for a dictionary of words. Each row of the dataset contains the part of speech, the word, the positive score, and the negative score for the word. A word may appear more than once if it can appear with different part of speech tags. 

These scores are designed so that we can also derive the *objectivity score* of the word from the positive and negative scores.

Objectivity is calculated: 

    1. - (positive_score + negative_score)

Thus if a score has zero positive score and negative score it is completely objective. If a score has, for example, 0.5 positive and 0.5 negative, it may not be any more positive than negative but we can tell that it is subjective (objectivity = 0.).


In [4]:
import pandas as pd
import numpy as np

In [5]:
sen = pd.read_csv('./datasets/sentiment_words_simple.csv')

In [6]:
len(sen)

155287

In [7]:
sen.head()

,pos,word,pos_score,neg_score
0,adj,.22-caliber,0.0,0.0
1,adj,.22-calibre,0.0,0.0
2,adj,.22_caliber,0.0,0.0
3,adj,.22_calibre,0.0,0.0
4,adj,.38-caliber,0.0,0.0


**Make the part of speech tags uppercase (this will come in handy later when we use Spacy).**

In [8]:
sen.pos = sen.pos.map(lambda x: x.upper())

<a id='adj-scores'></a>

### Engineer objectivity and positive difference scores

Since subjective vs. objective is embedded in the positive and negative scores, we should extract this and convert the positive and negative into a relative difference scores.

**Calculate two new scores:**

    objectivity = 1. - (pos_score + neg_score)
    pos_vs_neg = pos_score - neg_score
    

In [9]:
sen['objectivity'] = 1. - (sen.pos_score + sen.neg_score)
sen['pos_vs_neg'] = sen.pos_score - sen.neg_score

In [12]:
sen.head(3)

,pos,word,pos_score,neg_score,objectivity,pos_vs_neg
0,ADJ,.22-caliber,0.0,0.0,1.0,0.0
1,ADJ,.22-calibre,0.0,0.0,1.0,0.0
2,ADJ,.22_caliber,0.0,0.0,1.0,0.0


<a id='pos-dict'></a>

### Put scores in a part of speech dictionary

The dictionary format of the data will be much easier to index using our parsing functions later on. Create a dictionary where the keys are the four part of speech tags:

    ADJ
    NOUN
    VERB
    ADV

For each key, store a dictionary that contains all of the words for that part of speech with their objectivity and positive vs. negative scores.

In [13]:
sen_dict = {'ADJ':{},'NOUN':{},'VERB':{},'ADV':{}}

for i, row in enumerate(sen.itertuples()):
    if (i % 10000) == 0:
        print i
    sen_dict[row[1]][row[2]] = {'objectivity':row[5], 'pos_vs_neg':row[6]}


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000


<a id='rt-reviews'></a>

## Load the rotten tomatoes reviews dataset

---

This dataset has:
    
    critic: critic's name
    fresh: fresh vs. rotten rating
    imdb: code for imdb
    publication: where the review was published
    quote: the review snippet
    review_date: date of review
    rtid: rottentomatoes id
    title: name of movie

In [14]:
rt = pd.read_csv('./datasets/rt_critics.csv')

In [15]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,fresh,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story
2,David Ansen,fresh,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story
3,Leonard Klady,fresh,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story
4,Jonathan Rosenbaum,fresh,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story


<a id='subset'></a>

### Restrict data to reviews with valid ratings and reviews over 10 words long

Also clean up the reviews, making a column with the case and punctuation removed.

In [16]:
rt = rt[rt.fresh.isin(['fresh','rotten'])]
rt.fresh = rt.fresh.map(lambda x: 1 if x == 'fresh' else 0)

In [17]:
rt['quote_len'] = rt.quote.map(lambda x: len(x.split()))
rt = rt[rt.quote_len > 10]
rt.shape

(11215, 9)

In [18]:
import string
rt['qt'] = rt.quote.map(lambda x: unicode(''.join([y for y in list(x.lower()) if y in string.ascii_lowercase+" -'"])))
rt.qt = rt.qt.map(lambda x: x.replace('-',' '))

<a id='spacy'></a>

## Import spacy

---

The spacy package is the current gold standard for parsing the grammatical structure of text (aside from neural network architectures). We are going to use it to find the part of speech tags for the review words. 

Once we have parsed the tags with spacy, we can assign objectivity and valence scores by finding the match in our sentiment dataset.

In [19]:
import spacy
# python -m spacy download en
en_nlp = spacy.load('en')

**Parse a single quote:**

In [20]:
tmp = en_nlp(rt.qt.values[0])

In [25]:
tmp

so ingenious in concept design and execution that you could watch it on a postage stamp sized screen and still be engulfed by its charm

In [26]:
# we can get out single words with indexing:
tmp[3]

concept

**Print out the part of speech tags for each word in the quote:**

In [27]:
for token in tmp:
    print token, token.pos_

so ADV
ingenious ADJ
in ADP
concept NOUN
design NOUN
and CCONJ
execution NOUN
that ADJ
you PRON
could VERB
watch VERB
it PRON
on ADP
a DET
postage NOUN
stamp NOUN
sized VERB
screen NOUN
and CCONJ
still ADV
be VERB
engulfed VERB
by ADP
its ADJ
charm NOUN


<a id='multi'></a>
### Parse all the quotes using spacy's multithreaded parser

Parsing a lot of text can take quite awhile. Luckily spacy comes with multithreading functionality to speed up the process considerably. Below is code that will parse the quotes across multiple threads and assign them to a list.

In [28]:
parsed_quotes = []
for i, parsed in enumerate(en_nlp.pipe(rt.qt.values, batch_size=50, n_threads=4)):
    assert parsed.is_parsed
    if (i % 1000) == 0:
        print i
    parsed_quotes.append(parsed)        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


<a id='pos-features'></a>

## Create features with part of speech proportions

---

With our spacy parsed reviews, we have a lot of feature engineering potential even before we get to sentiment. Something simple we could do is calculate the proportion of words in the quote that have each part of speech tag. We can try using these as predictors in a model later.

**Find all the unique part of speech categories in the reviews.**

In [29]:
unique_pos = []
for parsed in parsed_quotes:
    unique_pos.extend([t.pos_ for t in parsed])
unique_pos = np.unique(unique_pos)
print unique_pos

[u'ADJ' u'ADP' u'ADV' u'CCONJ' u'DET' u'INTJ' u'NOUN' u'NUM' u'PART'
 u'PRON' u'PROPN' u'PUNCT' u'SPACE' u'SYM' u'VERB' u'X']


**Create the proportion columns for each part of speech.**

In [30]:
for pos in unique_pos:
    rt[pos+'_prop'] = 0.

**Iterate through the reviews and calculate the proportions of each part of speech tag.**

In [31]:
rt = rt.reset_index(drop=True)
for i, parsed in enumerate(parsed_quotes):
    if (i % 1000) == 0:
        print i
    parsed_len = len(parsed)
    for pos in unique_pos:
        count = len([x for x in parsed if x.pos_ == pos])
        rt.ix[i, pos+'_prop'] = float(count)/parsed_len

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [32]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,NOUN_prop,NUM_prop,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...,...,0.280000,0.0,0.000000,0.080000,0.0,0.0,0.0000,0.0,0.200000,0.0
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...,...,0.384615,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.230769,0.0
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...,...,0.277778,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.055556,0.0
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...,...,0.437500,0.0,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.125000,0.0
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...,...,0.302326,0.0,0.023256,0.046512,0.0,0.0,0.0000,0.0,0.139535,0.0


<a id='assign'></a>

## Assign sentiment scores
---

We will now use the parsed reviews and the sentiment dataset to assign the average objectivity and positive vs. negative scores.

If a word cannot be found in the dataset we can ignore it. If a review has no words that match something in our dataset, will can assign overall neutral scores of `objectivity = 1` and `pos_vs_neg = 0`.

There are definitely problems with this approach, but for now we can keep it "dumb" and see if things improve when we use the VADER analyzer later.

In [39]:
def scorer(parsed):
    obj_scores, pvn_scores = [], []
    for token in [t for t in parsed if t.pos_ in ['NOUN','VERB','ADV','ADJ']]:
        try:
            obj_scores.append(sen_dict[token.pos_][str(token)]['objectivity'])
            pvn_scores.append(sen_dict[token.pos_][str(token)]['pos_vs_neg'])
        except:
            pass
    if len(obj_scores) == 0:
        obj_scores = [1.]
    if len(pvn_scores) == 0:
        pvn_scores = [0.]
    return [obj_scores, pvn_scores]


scores = []
for i, parsed in enumerate(parsed_quotes):
    if (i % 1000) == 0:
        print i
    scores.append(scorer(parsed))
    
rt['objectivity_avg'] = [np.mean(x[0]) for x in scores]
rt['pos_vs_neg_avg'] = [np.mean(x[1]) for x in scores]


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [40]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop,objectivity_avg,pos_vs_neg_avg
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...,...,0.000000,0.080000,0.0,0.0,0.0000,0.0,0.200000,0.0,0.876202,0.061813
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.230769,0.0,1.000000,0.000000
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...,...,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.055556,0.0,0.840909,0.075284
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...,...,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.125000,0.0,0.850521,0.048438
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...,...,0.023256,0.046512,0.0,0.0,0.0000,0.0,0.139535,0.0,0.901589,0.053068


<a id='print-most'></a>
## Print out the most positive and most negative reviews
---

Now that we have the average valence for reviews, try printing out the top 10 most positive and top 10 most negative reviews to visually verify that our approach makes sense.

In [41]:
rt.sort_values('pos_vs_neg_avg', ascending=False, inplace=True)
for quote in rt.quote[0:10]:
    print quote
    print '============================================================\n'

Streep (the best thing she has done in ages) carries it along.

High Noon combines its points about good citizenship with some excellent picturemaking.

Paths of Glory is all about that greatest of all movie subjects: power.

Improbabilities and all, Simpatico still boasts wonderful scenes and a cast that is truly superb.

As bustling and impassioned as the best Sturges and Capra movies.

From Russia with Love is a preposterous, skillful slab of hardhitting, sexy hokum.

Succeeds, in part, because the film is as non-judgmental as the famed sex researcher himself.

It's an excellent movie for kids, because it is about how amazing children can be.

Is Dr. Strangelove Kubrick's best movie? Along with Paths of Glory, absolutely.

Hanks is superb, reemploying the childlike presence he brought to Big.



In [42]:
rt.sort_values('pos_vs_neg_avg', ascending=True, inplace=True)
for quote in rt.quote[0:10]:
    print quote
    print '============================================================\n'

What pulls you over the bum spots is the electrifying immediacy.

Unoriginal and insulting, 3 Strikes goes down without scoring a single chuckle.

Brooding, somber film is ragged around the edges and not without problematic aspects.

Its tone is never exactly comedic and its horrific touches are more disgusting than scary.

It's a disturbing, hopeless, irredeemable series of images that will scar you if you wander into it unprepared.

...Liar Liar stands to make a liar out of those who predicted that Carrey's career was on the skids.

A silly movie, with silly jokes and a silly story. But the talents at work in it are not silly.

Likely to be disappointing to Almodovar's admirers, and inexplicable to anyone else.

It retains the cheesy look of the 1979 original, pure schlock not gussied up to appear to be anything else.

Both bleak and bleakly funny, appalling in its excesses and exhilarating in its execution.



<a id='print-most-obj'></a>

## Print out the most objective and most subjective reviews
---

Do the same as above, but now sort by the objectivity. What kind of differences do you notice between these? Does our approach actually appear to capture meaningful subjectivity and objectivity in the reviews?

In [43]:
rt.sort_values('objectivity_avg', ascending=False, inplace=True)
for quote in rt.quote[0:10]:
    print quote
    print '============================================================\n'

Dr. Dolittle runs out of ideas long before the projector runs out of film.

The funniest movie of 1975 and probably the silliest movie ever made.

Either they had something on [Nick Gomez] or he needed the money.

Some of the funniest scenes bounce off the nightmares of every bride and groom before the wedding.

Everything about Something to Talk About feels off by a few beats.

By the end of the film, Selena has been all but canonized.

Magnolia makes it three-for-three for writer and director Paul Thomas Anderson

It plays like a movie dreamed up by people in a studio marketing department who'd decided to bare their souls.

Barbara Stanwyck is the sexiest con woman ever captured on film.

Offers an abundance of pleasures, especially in the realm of characterization and atmosphere.



In [44]:
rt.sort_values('objectivity_avg', ascending=True, inplace=True)
for quote in rt.quote[0:10]:
    print quote
    print '============================================================\n'

They keep getting worse and worse and worse . . .

At its best when it's being lighthearted and at its weakest when it takes a halfhearted stab at semi-seriousness.

At its best, it's a valentine of venom, sent with mirth and malice aforethought.

What pulls you over the bum spots is the electrifying immediacy.

I am not sure why this isn't very funny, but it's not.

Hawthorne is by turn outrageous and pathetic and imperious and poignant and very funny.

At its best, this achieves the beauty and grandeur of a Kurosawa epic -- at its worst, however, it feels like a Python remake of The Vikings.

In spite of its shortcomings, children love these characters and will enjoy Tigger.

Hilarious, sexy, clever, playful and as initially teasing as it is ultimately satisfying.

Jumps adroitly between the macho and anti-macho, the romantic and anti-romantic.



In [45]:
# There doesn't seem to be much signal in the objectivity score. They look pretty
# similar to me!

<a id='model'></a>

## Build a model to classify fresh vs. rotten with the sentiment and grammar features

---

Let's use the features we've created to construct a Logistic Regression to predict whether a review is fresh vs. rotten. 

Don't forget to check the baseline score, and it's a good practice to standardize your predictors.


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [61]:
X = rt[['objectivity_avg','pos_vs_neg_avg','quote_len']+[x for x in rt.columns if x.endswith('_prop')]]
y = rt.fresh.values

ss = StandardScaler()
Xs = ss.fit_transform(X)

lr_scores = cross_val_score(LogisticRegression(), Xs, y, cv=10)
print np.mean(lr_scores), rt.fresh.mean()
rt_new = rt.copy()
rt_new['estimation'] = rt.fresh.mean()
print r2_score(rt_new['estimation'], rt.fresh)
rt.fresh.value_counts()
6898.0 / (6898 + 4317)

0.640740894059 0.615069103879
0.0


0.6150691038787338

In [49]:
X.columns

Index([u'objectivity_avg',  u'pos_vs_neg_avg',       u'quote_len',
              u'ADJ_prop',        u'ADP_prop',        u'ADV_prop',
            u'CCONJ_prop',        u'DET_prop',       u'INTJ_prop',
             u'NOUN_prop',        u'NUM_prop',       u'PART_prop',
             u'PRON_prop',      u'PROPN_prop',      u'PUNCT_prop',
            u'SPACE_prop',        u'SYM_prop',       u'VERB_prop',
                u'X_prop'],
      dtype='object')

In [50]:
lr = LogisticRegression().fit(Xs, y)
for var, coef in zip(X.columns, lr.coef_[0]):
    print var, coef

objectivity_avg -0.126233159042
pos_vs_neg_avg 0.456829490129
quote_len 0.0972697524415
ADJ_prop 0.0673598722125
ADP_prop -0.00277071262457
ADV_prop -0.116993974421
CCONJ_prop 0.0659635231254
DET_prop -0.0393094564402
INTJ_prop -0.0148773028482
NOUN_prop 0.115966462762
NUM_prop 0.0297202568141
PART_prop -0.0700633312946
PRON_prop 0.0994019471933
PROPN_prop 0.0642794454372
PUNCT_prop -0.020160836971
SPACE_prop -0.00129654936462
SYM_prop 0.0122382782534
VERB_prop -0.155693928364
X_prop 0.0144606346576


In [51]:
# The coefficients make sense for the sentiment features.

In [52]:
from sklearn.linear_model import LogisticRegressionCV

# Try a lasso, find the best penalty.
lrcv = LogisticRegressionCV(penalty='l1', solver='liblinear', Cs=25, cv=10)
lrcv.fit(Xs, y)


LogisticRegressionCV(Cs=25, class_weight=None, cv=10, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [53]:
# looks like it still keeps a fair amount of the features.
lrcv.coef_

array([[-0.12168311,  0.45145997,  0.09190359,  0.06085636,  0.        ,
        -0.11661983,  0.06352011, -0.03760395, -0.01119044,  0.10467708,
         0.02422881, -0.06688845,  0.08826355,  0.05903704, -0.01672759,
         0.        ,  0.00727494, -0.15485324,  0.01022316]])

In [54]:
rt.head(2)

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop,objectivity_avg,pos_vs_neg_avg
5616,Mick LaSalle,0,120179.0,San Francisco Chronicle,They keep getting worse and worse and worse . . .,2000-01-01,12541.0,Speed 2: Cruise Control,11,they keep getting worse and worse and worse,...,0.0,0.111111,0.0,0.0,0.111111,0.0,0.222222,0.0,0.336648,-0.246352
5487,James Berardinelli,1,120032.0,ReelViews,At its best when it's being lighthearted and a...,2000-01-01,13484.0,Romy and Michele's High School Reunion,19,at its best when it's being lighthearted and a...,...,0.0,0.095238,0.0,0.0,0.000000,0.0,0.142857,0.0,0.383333,0.183333


<a id='vader'></a>

## Use the VADER library to get better sentiment scores
---

The [VADER](https://github.com/cjhutto/vaderSentiment) package for python is a more advanced way to calculate positivity, negativity, and objectivity in our reviews. The github page describes VADER as:

> VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.

You will likely need to install VADER with pip or conda. Instructions can be found on the github page. Once you have it installed you can load the `SentimentIntensityAnalyzer` and parse text.

**Parse a couple of quotes with the `SentimentIntensityAnalyzer` and print out the dictionary of scores using `analyzer.polarity_scores`:

In [63]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [64]:
analyzer = SentimentIntensityAnalyzer()
for sentence in rt.quote.values[0:2]:
    vs = analyzer.polarity_scores(sentence)
    print sentence
    print vs

They keep getting worse and worse and worse . . .
{'neg': 0.65, 'neu': 0.35, 'pos': 0.0, 'compound': -0.8519}
At its best when it's being lighthearted and at its weakest when it takes a halfhearted stab at semi-seriousness.
{'neg': 0.253, 'neu': 0.498, 'pos': 0.249, 'compound': -0.0258}


You can see that these scores look more legitimate. VADER polarity score dictionaries have 4 elements: `neg`, `pos`, `neu` and `compound`. The compound score is a single metric that represents the "overall" valence.

**Calculate the four scores for each review and save them as features in the dataframe.**

In [65]:
rt['vader_neg'] = 0
rt['vader_pos'] = 0
rt['vader_neu'] = 0
rt['vader_compound'] = 0

for i, q in enumerate(rt.quote.values):
    vs = analyzer.polarity_scores(q)
    rt.iloc[i, -4] = vs['neg']
    rt.iloc[i, -3] = vs['pos']
    rt.iloc[i, -2] = vs['neu']
    rt.iloc[i, -1] = vs['compound']

In [66]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,SPACE_prop,SYM_prop,VERB_prop,X_prop,objectivity_avg,pos_vs_neg_avg,vader_neg,vader_pos,vader_neu,vader_compound
5616,Mick LaSalle,0,120179.0,San Francisco Chronicle,They keep getting worse and worse and worse . . .,2000-01-01,12541.0,Speed 2: Cruise Control,11,they keep getting worse and worse and worse,...,0.111111,0.0,0.222222,0.0,0.336648,-0.246352,0.650,0.000,0.350,-0.8519
5487,James Berardinelli,1,120032.0,ReelViews,At its best when it's being lighthearted and a...,2000-01-01,13484.0,Romy and Michele's High School Reunion,19,at its best when it's being lighthearted and a...,...,0.000000,0.0,0.142857,0.0,0.383333,0.183333,0.253,0.249,0.498,-0.0258
1680,Michael Wilmington,1,106220.0,Chicago Tribune,"At its best, it's a valentine of venom, sent w...",2013-04-11,12634.0,Addams Family Values,14,at its best it's a valentine of venom sent wit...,...,0.000000,0.0,0.133333,0.0,0.425000,-0.075000,0.000,0.415,0.585,0.8316
1427,Peter Travers,1,106339.0,Rolling Stone,What pulls you over the bum spots is the elect...,2007-09-10,15730.0,Backbeat,11,what pulls you over the bum spots is the elect...,...,0.000000,0.0,0.181818,0.0,0.427083,-0.364583,0.000,0.000,1.000,0.0000
2161,Roger Ebert,0,111127.0,Chicago Sun-Times,"I am not sure why this isn't very funny, but i...",2000-01-01,13696.0,Serial Mom,12,i am not sure why this isn't very funny but it...,...,0.000000,0.0,0.214286,0.0,0.472222,-0.256656,0.268,0.000,0.732,-0.3165


<a id='vader-model'></a>

### Fit a model using the VADER sentiment features

Does this model perform better? 

In [58]:
X = rt[['vader_neg','vader_pos','vader_neu','vader_compound','quote_len']]
y = rt.fresh.values

Xs = StandardScaler().fit_transform(X)

scores = cross_val_score(LogisticRegression(), Xs, y, cv=10)
print scores
print np.mean(scores)

# We do slightly better. I've also left out the part of speech stuff so that has an impact too.

[ 0.67290553  0.67023173  0.65240642  0.6684492   0.67914439  0.63636364
  0.6372549   0.63336307  0.61785714  0.58571429]
0.645369029049


In [59]:
for c, v in zip(LogisticRegression().fit(Xs, y).coef_[0], ['neg','pos','neu','compound','len']):
    print v, c
    
# All the coefficients make sense.

neg -0.25445510214
pos 0.33708994887
neu -0.129669341392
compound 0.159047475754
len 0.103301240988


<a id='vader-top'></a>

### Print out the top most negative, positive, neutral, and subjective features by VADER score

In [60]:
rt.sort_values('vader_neg', ascending=False, inplace=True)
for i in range(5):
    print rt.quote.values[i]
    print '---------------------------------------------------\n'

I hated this movie. Hated hated hated hated hated this movie. Hated it.
---------------------------------------------------

They keep getting worse and worse and worse . . .
---------------------------------------------------

What's the fourth "Die Hard" called? I keep forgetting. "Die Hard: With a Pension"? "Die Hardened Arteries"? "Die Laughing"?
---------------------------------------------------

A shambolic, deafening, intelligence-insulting mess, a crushing failure on almost all counts.
---------------------------------------------------

So distressingly dark, grim, and cynical it's likely to make kids cry!
---------------------------------------------------



In [61]:
rt.sort_values('vader_pos', ascending=False, inplace=True)
for i in range(5):
    print rt.quote.values[i]
    print '---------------------------------------------------\n'

A truly funny, sophisticated, compassionate, mainstream Hollywood comedy about very modern homosexuality.
---------------------------------------------------

Extremely handsome production values and a great supporting cast round out the virtues.
---------------------------------------------------

For lovers of romantic comedies through the ages, Roman Holiday remains a favorite.
---------------------------------------------------

The Karate Kid exhibits warmth and friendly, predictable humor, its greatest assets.
---------------------------------------------------

Charming performances and easygoing humour are the strengths of Jones's enjoyable Oirish romp.
---------------------------------------------------



In [62]:
rt.sort_values('vader_neu', ascending=False, inplace=True)
for i in range(5):
    print rt.quote.values[i]
    print '---------------------------------------------------\n'

Dr. Dolittle runs out of ideas long before the projector runs out of film.
---------------------------------------------------

Visually a macabre knockout, this 75-minute fantasy boasts some of the wittiest, most vigorous stop-motion animation effects in the history of the process.
---------------------------------------------------

The uneven flow is as likely to lead to a snoozing viewer as to one on the edge of their seat.
---------------------------------------------------

The plot is involving, especially as it builds to its seemingly impossible-to-solve finale.
---------------------------------------------------

Offers a departure (albeit not a radical one) from the norm.
---------------------------------------------------



In [63]:
rt.sort_values('vader_neu', ascending=True, inplace=True)
for i in range(5):
    print rt.quote.values[i]
    print '---------------------------------------------------\n'

I hated this movie. Hated hated hated hated hated this movie. Hated it.
---------------------------------------------------

Welcome to Natural Born Killers, Stone's empty, manic meditation on society's glorification of violence and the ugly heroes it loves to hate.
---------------------------------------------------

Branagh's lame stab at a romantic psychological thriller makes no sense.
---------------------------------------------------

A truly funny, sophisticated, compassionate, mainstream Hollywood comedy about very modern homosexuality.
---------------------------------------------------

Hilarious, sexy, clever, playful and as initially teasing as it is ultimately satisfying.
---------------------------------------------------

